# Input pipeline into Keras

In this notebook, we will look at how to read large datasets, datasets that may not fit into memory, using TensorFlow. We can use the tf.data pipeline to feed data to Keras models that use a TensorFlow backend.

Let's start off with the Python imports that we need.

In [24]:
import os, json, math
import numpy as np
import tensorflow as tf
tf.enable_v2_behavior()
print(tf.__version__)

2.0.0-alpha0


## Locating the CSV files

We will start with the CSV files that we wrote out in the [first notebook](../01_explore/taxifare.iypnb) of this sequence. Just so you don't have to run the notebook, we saved a copy in ../data

In [25]:
!ls -l ../data/*.csv

-rw-r--r-- 1 jupyter jupyter 126266 Jun  3 15:48 ../data/taxi-test.csv
-rw-r--r-- 1 jupyter jupyter 593612 Jun  3 15:48 ../data/taxi-train.csv
-rw-r--r-- 1 jupyter jupyter 126833 Jun  3 15:48 ../data/taxi-valid.csv


## Use tf.data to read the CSV files

See the documentation for [make_csv_dataset](https://www.tensorflow.org/api_docs/python/tf/data/experimental/make_csv_dataset).
If you have TFRecords (which is recommended), use [make_batched_features_dataset](https://www.tensorflow.org/api_docs/python/tf/data/experimental/make_batched_features_dataset) instead.

In [42]:
CSV_COLUMNS  = ['fare_amount',  'pickup_datetime',
                'pickup_longitude', 'pickup_latitude', 
                'dropoff_longitude', 'dropoff_latitude', 
                'passenger_count', 'key']
LABEL_COLUMN = 'fare_amount'
DEFAULTS     = [[0.0],['na'],[0.0],[0.0],[0.0],[0.0],[0.0],['na']]

In [43]:
# load the training data
def load_dataset(pattern):
  return tf.data.experimental.make_csv_dataset(pattern, 1, CSV_COLUMNS, DEFAULTS)

tempds = load_dataset('../data/taxi-train*')
print(tempds)

<PrefetchDataset shapes: OrderedDict([(fare_amount, (1,)), (pickup_datetime, (1,)), (pickup_longitude, (1,)), (pickup_latitude, (1,)), (dropoff_longitude, (1,)), (dropoff_latitude, (1,)), (passenger_count, (1,)), (key, (1,))]), types: OrderedDict([(fare_amount, tf.float32), (pickup_datetime, tf.string), (pickup_longitude, tf.float32), (pickup_latitude, tf.float32), (dropoff_longitude, tf.float32), (dropoff_latitude, tf.float32), (passenger_count, tf.float32), (key, tf.string)])>


Note that this is a prefetched dataset. If you loop over the dataset, you'll get the rows one-by-one. Let's convert each row into a Python dictionary:

In [44]:
# print a few of the rows
for n, data in enumerate(tempds):
    row_data = {k: v.numpy() for k,v in data.items()}
    print(n, row_data)
    if n > 2:
        break

0 {'dropoff_longitude': array([-73.97569], dtype=float32), 'passenger_count': array([1.], dtype=float32), 'fare_amount': array([6.9], dtype=float32), 'pickup_longitude': array([-73.97808], dtype=float32), 'dropoff_latitude': array([40.780483], dtype=float32), 'pickup_datetime': array([b'2009-10-27 20:59:45+00:00'], dtype=object), 'pickup_latitude': array([40.76386], dtype=float32), 'key': array([b'5487'], dtype=object)}
1 {'dropoff_longitude': array([-73.96535], dtype=float32), 'passenger_count': array([1.], dtype=float32), 'fare_amount': array([15.], dtype=float32), 'pickup_longitude': array([-73.96879], dtype=float32), 'dropoff_latitude': array([40.801003], dtype=float32), 'pickup_datetime': array([b'2015-02-03 22:32:35+00:00'], dtype=object), 'pickup_latitude': array([40.75741], dtype=float32), 'key': array([b'3025'], dtype=object)}
2 {'dropoff_longitude': array([-73.95738], dtype=float32), 'passenger_count': array([1.], dtype=float32), 'fare_amount': array([4.5], dtype=float32), 'p

What we really need is a dictionary of features + a label. So, we have to do two things to the above dictionary. (1) remove the unwanted column "key" and (2) keep the label separate from the features.

In [45]:
# get features, label
def features_and_labels(row_data):
    for unwanted_col in ['pickup_datetime', 'key']:
        row_data.pop(unwanted_col)
    label = row_data.pop(LABEL_COLUMN)
    return row_data, label  # features, label

# print a few rows to make it sure works
for n, data in enumerate(tempds):
    row_data = {k: v.numpy() for k,v in data.items()}
    features, label = features_and_labels(row_data)
    print(n, label, features)
    if n > 2:
        break

0 [6.9] {'dropoff_longitude': array([-73.97569], dtype=float32), 'passenger_count': array([1.], dtype=float32), 'pickup_longitude': array([-73.97808], dtype=float32), 'dropoff_latitude': array([40.780483], dtype=float32), 'pickup_latitude': array([40.76386], dtype=float32)}
1 [15.] {'dropoff_longitude': array([-73.96535], dtype=float32), 'passenger_count': array([1.], dtype=float32), 'pickup_longitude': array([-73.96879], dtype=float32), 'dropoff_latitude': array([40.801003], dtype=float32), 'pickup_latitude': array([40.75741], dtype=float32)}
2 [4.5] {'dropoff_longitude': array([-73.95738], dtype=float32), 'passenger_count': array([1.], dtype=float32), 'pickup_longitude': array([-73.9507], dtype=float32), 'dropoff_latitude': array([40.77518], dtype=float32), 'pickup_latitude': array([40.785786], dtype=float32)}
3 [10.5] {'dropoff_longitude': array([-73.96096], dtype=float32), 'passenger_count': array([1.], dtype=float32), 'pickup_longitude': array([-73.982216], dtype=float32), 'dropof

## Batching

Let's do both (loading, features_label)
in our load_dataset function, and also add batching.

In [46]:
def load_dataset(pattern, batch_size):
  return (
      tf.data.experimental.make_csv_dataset(pattern, batch_size, CSV_COLUMNS, DEFAULTS)
             .map(features_and_labels) # features, label
  )

# try changing the batch size and watch what happens.
tempds = load_dataset('../data/taxi-train*', batch_size=2)
print(list(tempds.take(3))) # truncate and print as a list 

[(OrderedDict([('pickup_longitude', <tf.Tensor: id=3384, shape=(2,), dtype=float32, numpy=array([-74.009514, -73.99487 ], dtype=float32)>), ('pickup_latitude', <tf.Tensor: id=3383, shape=(2,), dtype=float32, numpy=array([40.72891 , 40.739902], dtype=float32)>), ('dropoff_longitude', <tf.Tensor: id=3381, shape=(2,), dtype=float32, numpy=array([-74.010635, -73.99037 ], dtype=float32)>), ('dropoff_latitude', <tf.Tensor: id=3380, shape=(2,), dtype=float32, numpy=array([40.72731, 40.74638], dtype=float32)>), ('passenger_count', <tf.Tensor: id=3382, shape=(2,), dtype=float32, numpy=array([5., 1.], dtype=float32)>)]), <tf.Tensor: id=3385, shape=(2,), dtype=float32, numpy=array([15.3,  4.5], dtype=float32)>), (OrderedDict([('pickup_longitude', <tf.Tensor: id=3390, shape=(2,), dtype=float32, numpy=array([-73.95611 , -74.004395], dtype=float32)>), ('pickup_latitude', <tf.Tensor: id=3389, shape=(2,), dtype=float32, numpy=array([40.767582, 40.719425], dtype=float32)>), ('dropoff_longitude', <tf.Te

## Shuffling

When training a deep learning model in batches over multiple workers, it is helpful if we shuffle the data. That way, different workers will be working on different parts of the input file at the same time, and so averaging gradients across workers will help. Also, during training, we will need to read the data indefinitely.

In [47]:
def load_dataset(pattern, batch_size=1, mode=tf.estimator.ModeKeys.EVAL):
  dataset = (tf.data.experimental.make_csv_dataset(pattern, batch_size, CSV_COLUMNS, DEFAULTS)
             .map(features_and_labels) # features, label
             .cache())
  if mode == tf.estimator.ModeKeys.TRAIN:
        dataset = dataset.shuffle(1000).repeat()
  dataset = dataset.prefetch(1) # take advantage of multi-threading; 1=AUTOTUNE
  return dataset

tempds = load_dataset('../data/taxi-train*', 2, tf.estimator.ModeKeys.TRAIN)
print(list(tempds.take(1)))
tempds = load_dataset('../data/taxi-valid*', 2, tf.estimator.ModeKeys.EVAL)
print(list(tempds.take(1)))

[(OrderedDict([('pickup_longitude', <tf.Tensor: id=3516, shape=(2,), dtype=float32, numpy=array([-74.00582, -73.96021], dtype=float32)>), ('pickup_latitude', <tf.Tensor: id=3515, shape=(2,), dtype=float32, numpy=array([40.73631 , 40.781963], dtype=float32)>), ('dropoff_longitude', <tf.Tensor: id=3513, shape=(2,), dtype=float32, numpy=array([-73.993866, -73.95238 ], dtype=float32)>), ('dropoff_latitude', <tf.Tensor: id=3512, shape=(2,), dtype=float32, numpy=array([40.74126 , 40.777256], dtype=float32)>), ('passenger_count', <tf.Tensor: id=3514, shape=(2,), dtype=float32, numpy=array([3., 1.], dtype=float32)>)]), <tf.Tensor: id=3517, shape=(2,), dtype=float32, numpy=array([4.9, 5.5], dtype=float32)>)]
[(OrderedDict([('pickup_longitude', <tf.Tensor: id=3618, shape=(2,), dtype=float32, numpy=array([-73.97676, -73.94965], dtype=float32)>), ('pickup_latitude', <tf.Tensor: id=3617, shape=(2,), dtype=float32, numpy=array([40.788414, 40.772587], dtype=float32)>), ('dropoff_longitude', <tf.Tenso

In the next notebook, we will build the model using this input pipeline.

Copyright 2019 Google Inc.
Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at
http://www.apache.org/licenses/LICENSE-2.0
Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.